<a href="https://colab.research.google.com/github/vyperid/Qdrant-vs-ChromaDB/blob/main/QdrantOpenAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q PyMuPDF
!pip install -q langchain-text-splitters
!pip install -q qdrant-client
!pip install -q langchain
!pip install -q openai
!pip install -q langchain_community
!pip install -q tiktoken

In [2]:
import pymupdf
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [3]:
def pdf_to_txt(pdf_path, output_path):
  document = pymupdf.open(pdf_path)

  text = ""

  for page in range(len(document)):
    page = document.load_page(page)
    text += page.get_text()

  with open(output_path, "w", encoding='utf-8') as f:
    f.write(text)

pdf_path = '/content/62f12134d4d3a-dead-poets-society.pdf'
txt_path = 'deadpoets.txt'
pdf_to_txt(pdf_path, txt_path)

In [4]:
with open("/content/deadpoets.txt") as f:
  document = f.read()

In [5]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=100,
    length_function=len,
    is_separator_regex=True,
    separators=["\."]
)

In [6]:
text_chunks = text_splitter.split_text(document)

In [16]:
import os
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings
from qdrant_client import models, QdrantClient
import time
from qdrant_client.http import models
from langchain.vectorstores import Qdrant

In [8]:
os.environ["OPENAI_API_KEY"] = ""

In [9]:
embedding_function = OpenAIEmbeddings()

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [13]:
qdrant = QdrantClient(":memory:")

In [14]:
qdrant.recreate_collection(
    collection_name = "deadpoetssociety",
    vectors_config = models.VectorParams(
        size=1536,
        distance=models.Distance.COSINE)
)

<ipython-input-14-f5c86d4388d3>:1: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant.recreate_collection(


True

In [ ]:
embedding = OpenAIEmbeddings()

vector_store = Qdrant(
    client = qdrant,
    collection_name = "deadpoetssociety",
    embeddings = embedding
)

vector_store.add_texts(text_chunks)

In [26]:
def get_results(query):
  query_embedding = embedding.embed_query(query)

  results = qdrant.search(
      collection_name="deadpoetssociety",
      query_vector=query_embedding,
      query_filter=None,
      limit=1
  )
  return results

In [27]:
questions = [
    "Who is going to be Neil Perry's roommate?",
    "What did Todd do when Neil left the room?",
    "What was the Dead Poets Society according to Mr. Keating?",
    "How did the juniors feel about their first physical education class?",
    "What is Todd’s reaction to the pressure of attending Welton Academy?",
    "How does Mr. Keating’s unconventional approach to teaching Latin influence the students?",
    "Describe the atmosphere and activities during the Welton Academy’s opening ceremony.",
    "How does the character of Neil Perry evolve through his interactions with his father, and how does this affect his decisions and actions at Welton Academy?",
    "How does Mr. Keating’s philosophy of “Carpe Diem” challenge the traditional values taught at Welton Academy, and what are the implications for the students who embrace this philosophy?",
    "Examine how the theme of conformity versus individuality is portrayed through the interactions between Mr. Keating and the students, particularly in the context of Welton Academy’s values."
]

In [29]:
for question in questions:
  start = time.time()
  results = get_results(question)
  end = time.time()
  print(f"Time taken: {end - start}")
  print(results)

Time taken: 0.14249801635742188
[ScoredPoint(id='9f614d2cf4e7441f881654cab330fefe', version=0, score=0.8489721572085389, payload={'page_content': '.\n“I hear we’re going to be roommates,” he said. “I’m Neil\nPerry.”\n“Todd Anderson,” he replied softly. The boys walked in\nawkward silence.\n“Why’d you leave Balincrest?” Neil asked.\n“My brother went here.”\nNeil shook his head. “Oh, so you’re THAT Anderson.”\nTodd shrugged and groaned', 'metadata': None}, vector=None, shard_key=None, order_value=None)]
Time taken: 0.11868548393249512
[ScoredPoint(id='1103c30f71194117808bda73a6b6ebc2', version=0, score=0.8825464034159893, payload={'page_content': '. Neil looked at the photo and noticed\nthat Todd was slightly apart from the family group, with them\nbut not really a part of them. Todd then unpacked an engraved\nleather desk set and laid it out on his desk.\nNeil plopped on his bed and leaned against the headboard', 'metadata': None}, vector=None, shard_key=None, order_value=None)]
Time ta